In [27]:
import pickle
from flask import Flask, jsonify, request,render_template,Response
import requests
import warnings
warnings.filterwarnings('ignore')

In [28]:
data = pickle.load(open("../upload/pklfile/test_data_2.pkl",'rb'))
lr   = pickle.load(open('../upload/lgbmmodel.sav','rb'))
product_name_dict = pickle.load(open('../upload/pklfile/producIdName.pkl','rb'))  
app = Flask(__name__)

In [29]:
@app.route('/')
def index():
    return render_template('index.html')

In [30]:
@app.route('/login')
def login():
    return render_template('login.html')

In [31]:
@app.route('/predict', methods=['POST'])
def predict():
    to_predict_list = [int(x) for x in request.form.values()]
    order_number = to_predict_list[0]
    data1 = data[data.order_id==order_number]
    prediction = lr.predict(data1.drop(['order_id','user_id','product_id'],axis=1))
    data1['prediction'] = prediction
    data1 = data1[data1.prediction==1]
    output = [] 
    k=0
    # // predict add to array
    for i in data1.product_id:
        if i=='None':
            output.append("none")
            break
        else:
            k+=1
            product=product_name_dict[i].split()
            if(len(product)<=3):
                output.append(product_name_dict[i])
        if(k==6):
            break
        # fetching api
    api_key = '33171502-91ed4031c1cfc917923c7c5d3'
    last_img= []
    for k in output:
        text=k.split()
        last_txt=text[len(text)-1]
        url = f'https://pixabay.com/api/?key={api_key}&q={last_txt}&image_type=photo&per_page=10'
        response = requests.get(url)
        img_data = response.json()
        images = [hit['webformatURL'] for hit in img_data['hits']]
        if(len(images)!=0):
            last_img.append(images[2])   

        items = []
        for i in range(len(last_img)):
            item = {'image': last_img[i], 'name': output[i]}
            items.append(item)

            
    if len(output)<=0:
        return Response(render_template('output.html',data=['Banana','Dairy Milk','Carrot']))
    else:
        return Response(render_template('output.html',items=items))

In [32]:
if __name__ == '__main__':
    #app.debug = True
    app.run(host='0.0.0.0', port=8080)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.43.26:8080/ (Press CTRL+C to quit)
192.168.43.26 - - [19/Feb/2023 20:09:59] "GET /login HTTP/1.1" 200 -
192.168.43.26 - - [19/Feb/2023 20:10:31] "POST /predict HTTP/1.1" 200 -
